In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cpu


In [14]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)


['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [15]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])


In [16]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)



inputs:
tensor([[66, 62, 67, 58,  9,  1, 59, 68],
        [69, 65, 58, 54, 72, 54, 67, 73],
        [74, 72, 11,  1, 47, 68, 74, 65],
        [62, 67, 73, 68,  1, 73, 61, 58]])
targets:
tensor([[62, 67, 58,  9,  1, 59, 68, 71],
        [65, 58, 54, 72, 54, 67, 73, 65],
        [72, 11,  1, 47, 68, 74, 65, 57],
        [67, 73, 68,  1, 73, 61, 58,  1]])


In [17]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [18]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



g5N!﻿
 3*RDQl?Eirtewy﻿Bj&rAFM4S]eNfTmg?UP-l75(eX5*FJ(ox JGQf'7EP_te]*f﻿BNLZ800fE,l,p_BTIdK[(FY'Wd--?nmNQu:T3j!:t)3mgA:6PK*Wb&WZ6LwD)LN(u;cIcxaH?te_eyR?tf
0QfkOINL0BxkQ"0E2qi;t
k2FmfnGxtxwvQM,:stL&﻿H_rp(0rACjgph7*ZUkUP
J5JEBSrjD)5Zu)HgSKVNf?Ru:sBIQy?II["&.ZHuRnN_Jtz!(S77ZlpZw.
eEO)MceRLE4Zui﻿i.8z4n04df"-6j6sd0JG,1(Gr;teefZ80k2)y:qtE2Z[Z;a﻿O4n?RG"EZI)Qaa'e!mx
PTshFj2qVaKX41V3"jef2'!4*:'vQIls&
Y.I5.0eyxtnr 
6[mxQrU(-b'ACNTGzvTvIKht H﻿Jl?k&J7q[AfEg[i﻿?]uS.)-﻿jpz0fSX4rG&(RZ;[sB ![iIfEO8(J[p?Tv(w46Z]y


In [19]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.826, val loss: 4.787
step: 250, train loss: 4.737, val loss: 4.730
step: 500, train loss: 4.677, val loss: 4.684
step: 750, train loss: 4.616, val loss: 4.606
4.779698371887207


***need to familiarize audience with optimizers (AdamW, Adam, SGD, MSE…) no need to jump into the formulas, just what the optimizer does for us and some of the differences/similarities between them***

1. **Mean Squared Error (MSE)**: MSE is a common loss function used in regression problems, where the goal is to predict a continuous output. It measures the average squared difference between the predicted and actual values, and is often used to train neural networks for regression tasks.
2. **Gradient Descent (GD):**  is an optimization algorithm used to minimize the loss function of a machine learning model. The loss function measures how well the model is able to predict the target variable based on the input features. The idea of GD is to iteratively adjust the model parameters in the direction of the steepest descent of the loss function
3. **Momentum**: Momentum is an extension of SGD that adds a "momentum" term to the parameter updates. This term helps smooth out the updates and allows the optimizer to continue moving in the right direction, even if the gradient changes direction or varies in magnitude. Momentum is particularly useful for training deep neural networks.
4. **RMSprop**: RMSprop is an optimization algorithm that uses a moving average of the squared gradient to adapt the learning rate of each parameter. This helps to avoid oscillations in the parameter updates and can improve convergence in some cases.
5. **Adam**: Adam is a popular optimization algorithm that combines the ideas of momentum and RMSprop. It uses a moving average of both the gradient and its squared value to adapt the learning rate of each parameter. Adam is often used as a default optimizer for deep learning models.
6. **AdamW**: AdamW is a modification of the Adam optimizer that adds weight decay to the parameter updates. This helps to regularize the model and can improve generalization performance. We will be using the AdamW optimizer as it best suits the properties of the model we will train in this video.

find more optimizers and details at torch.optim

In [20]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


drt (jHxx RL2'a
eK[K0 s42!(jk2YQ9﻿Y10k--b&8qZ"﻿?KC3ebfTvG,kdjpN!kWKB"jBxr"oq;-﻿N36qf?y,apI65bFZj
M8bhKCWrrW)4uL.FCUGvpl-s(ezHD[hG80*LNyDT;&d21mxFspH_4]wVm1ms1qr;W_rvGNSHu8zC&ZRTB6
6s12E9YqVPZ"Dxw5MK.hNT;7z)5l -W:TUh!﻿YmZ"i8L?Uoh oSMG&-d*dxiE[JbOYJa&"?Kz*e0P&UnsR6PpQ:6stflc":XkT4M_iI[e"jB(RL-tefVA﻿MNic3t,"[D)lKW*pM'W_w06?X1kO]jb2zhtioOc'&J4vBQ﻿?WrRL&ti9AC_tn3.z?KWzhpQ085Q:&3S91;[m!:Tv6SSx JiE6RpnA&PDpoPV!HtjUJ;Nro)5dO0POtAF1dRhhfEZ"aGGnWx﻿ oRTyNF&JZud)q﻿p,dl1;l[Z"-b3M0fw)BM)yfvks:sGA4vtqH0&EPY)2K
